In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *

from pyspark.sql.types import *

In [3]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('Projetas_AIR_CIA')
    .getOrCreate()
)

In [16]:
df = spark.read.csv('E:\Desafio Projetas\AIR_CIA\AIR_CIA\ANAC_20211220_203627.csv', sep=';', header=True, inferSchema=True)
df.show()



+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|        Razão Social|ICAO IATA|              CNPJ|   Atividades Aéreas|       Endereço Sede|            Telefone|              E-Mail| Decisão Operacional|Data Decisão Operacional|Validade Operacional|
+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|ABSA - AEROLINHAS...|   LTG M3|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              10/02/2021|       

In [17]:
df.printSchema()

root
 |-- Razão Social: string (nullable = true)
 |-- ICAO IATA: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- Atividades Aéreas: string (nullable = true)
 |-- Endereço Sede: string (nullable = true)
 |-- Telefone: string (nullable = true)
 |-- E-Mail: string (nullable = true)
 |-- Decisão Operacional: string (nullable = true)
 |-- Data Decisão Operacional: string (nullable = true)
 |-- Validade Operacional: string (nullable = true)



In [18]:
df.toPandas().isna().sum()

Razão Social                0
ICAO IATA                   0
CNPJ                        0
Atividades Aéreas           0
Endereço Sede               0
Telefone                    0
E-Mail                      2
Decisão Operacional         0
Data Decisão Operacional    0
Validade Operacional        4
dtype: int64

In [19]:
df.columns

['Razão Social',
 'ICAO IATA',
 'CNPJ',
 'Atividades Aéreas',
 'Endereço Sede',
 'Telefone',
 'E-Mail',
 'Decisão Operacional',
 'Data Decisão Operacional',
 'Validade Operacional']

In [20]:
for coluna in df.columns:
    print(coluna, df.filter(df[coluna].isNull()).count())

Razão Social 0
ICAO IATA 0
CNPJ 0
Atividades Aéreas 0
Endereço Sede 0
Telefone 0
E-Mail 2
Decisão Operacional 0
Data Decisão Operacional 0
Validade Operacional 4


In [26]:
df = df.withColumnRenamed('Razão Social', 'razão_social')\
.withColumnRenamed('ICAO IATA', 'icao_iata')\
.withColumnRenamed('CNPJ', 'cnpj')\
.withColumnRenamed('Atividades Aéreas', 'atividades_aéreas')\
.withColumnRenamed('Endereço Sede', 'endereço_sede')\
.withColumnRenamed('Telefone', 'telefone')\
.withColumnRenamed('E-Mail', 'e_mail')\
.withColumnRenamed('Decisão Operacional', 'decisão_operacional')\
.withColumnRenamed('Data Decisão Operacional', 'data_decisão_operacional')\
.withColumnRenamed('Validade Operacional', 'validade_operacional')\

df.show(5)

+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|        razão_social|icao_iata|              cnpj|   atividades_aéreas|       endereço_sede|            telefone|              e_mail| decisão_operacional|data_decisão_operacional|validade_operacional|
+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+
|ABSA - AEROLINHAS...|   LTG M3|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO Nº 282|              10/02/2021|       

In [29]:
df.show (truncate = 0)

+------------------------------------------------------------------+---------+------------------+--------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+-----------------------------------+----------------------------+------------------------+--------------------+
|razão_social                                                      |icao_iata|cnpj              |atividades_aéreas                                                                           |endereço_sede                                                                                                                                                       |telefone                                     |e_mail                             |decisão_operacional         |data_decisão_operacio

## Separar a coluna 'ICAO IATA' em duas colunas, seu conteúdo está separado por espaço e pode não conter o código IATA, caso não contenha o código IATA, deixe o valor nulo.

In [30]:
df.select('icao_iata').show()

+---------+
|icao_iata|
+---------+
|   LTG M3|
|   ASO 2S|
|   SUL 0A|
|      ABJ|
|   ACN 2F|
|   AZU AD|
|   GLO G3|
|   PAM 7M|
|   MWM WD|
|   PTB 2Z|
|   SID 0S|
|   TAM JJ|
|   TTL L1|
+---------+



In [33]:
import pyspark


In [34]:
split_cols = pyspark.sql.functions.split(df['icao_iata'], ' ')

In [36]:
df1 = df.withColumn('icao', split_cols.getItem(0)) \
    .withColumn('iata', split_cols.getItem(1))

In [37]:
df1.show()

+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+----+
|        razão_social|icao_iata|              cnpj|   atividades_aéreas|       endereço_sede|            telefone|              e_mail| decisão_operacional|data_decisão_operacional|validade_operacional|icao|iata|
+--------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------------+--------------------+----+----+
|ABSA - AEROLINHAS...|   LTG M3|00.074.635/0001-33|TRANSPORTE AÉREO ...|AEROPORTO INTERNA...|      (11) 5582-8055|      gar@tam.com.br|       DECISÃO Nº 41|              22/04/2015|          23/04/2025| LTG|  M3|
|AEROSUL TÁXI AÉRE...|   ASO 2S|27.315.694/0001-02|SERVIÇOS AÉREOS P...|RODOVIA PR 218, K...|      (43) 3176-4030|operacoes@aerosul...|      DECISÃO

In [42]:
df1.select('iata', 'icao').show()

+----+----+
|iata|icao|
+----+----+
|  M3| LTG|
|  2S| ASO|
|  0A| SUL|
|null| ABJ|
|  2F| ACN|
|  AD| AZU|
|  G3| GLO|
|  7M| PAM|
|  WD| MWM|
|  2Z| PTB|
|  0S| SID|
|  JJ| TAM|
|  L1| TTL|
+----+----+

